# Imports

In [ ]:
import numpy as np
import random
import sys, os
import cv2
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, GlobalAveragePooling2D
from keras.layers import Conv2D, MaxPooling2D
import keras.losses
import keras.optimizers
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Convolution2D, MaxPooling2D, Flatten, BatchNormalization
from keras.applications import MobileNetV2
from keras.preprocessing import image_dataset_from_directory
import subprocess, os, shutil

# Mounting drive

In [ ]:
import subprocess, os, shutil
subprocess.run(['apt-get', 'install', 'unzip'])


In [ ]:
#from google.colab import drive
#drive.mount('drive')

In [ ]:

#shutil.copytree('./drive/My Drive/itl/itl_dataset', './itl_dataset')
for folder in ['itl_dataset', 'output_proper', 'output_bbox', 'output_proper', 'output_color']:
  if os.path.exists(folder):
    shutil.rmtree(folder)
print(os.getcwd())
VERSION=5
subprocess.run(['wget', f'http://students.mimuw.edu.pl/~mt406390/machine_learning/itl_dataset_v{VERSION}.zip'])
subprocess.run(['unzip', f'itl_dataset_v{VERSION}.zip'], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

# Loading data

In [ ]:
POINTS_PER_BOX=4
BBOXES=1
OUT_SIZE=POINTS_PER_BOX*BBOXES
INPUT_SHAPE=(224, 224, 1)

In [ ]:
import json
with open('annotations.json', 'r') as flabels:
  annots = json.load(flabels)

In [ ]:
dataset_size = len(annots)
print(dataset_size)
operators = np.zeros((dataset_size, OUT_SIZE))
print(operators.shape)
paths = [None] * dataset_size
for j, a in enumerate(annots):
  idx = int(a['name'][len('eq') : a['name'].find('.')])
  assert(j == idx)
  operators[idx] = np.array(a['op'])
  paths[idx] = os.path.join('output_proper', a['name'])




In [ ]:
from google.colab.patches import cv2_imshow

def load_img(path):
  return 255-cv2.cvtColor(imread(path), cv2.COLOR_BGR2GRAY).reshape(INPUT_SHAPE)



In [ ]:
from skimage.io import imread
from skimage.transform import resize
import numpy as np
import math
from keras.utils import Sequence



class ITLSequence(Sequence):

    def __init__(self, x_set, y_set, batch_size):
        self.x, self.y = x_set, y_set
        self.batch_size = batch_size

    def __len__(self):
        return math.ceil(len(self.x) / self.batch_size)

    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size:(idx + 1) *
        self.batch_size]
        batch_y = self.y[idx * self.batch_size:(idx + 1) *
        self.batch_size]
        #batch_y = np.ones((self.batch_size, 4))*100
        return np.array([
            load_img(file_name) for file_name in batch_x], dtype=np.float64), batch_y

In [ ]:
itlseq = ITLSequence(paths,operators, 64)

# Model


In [ ]:
def get_model(verbose=False):
    model = Sequential()   
    model.add(keras.applications.MobileNetV2(weights=None, include_top=False, input_shape=INPUT_SHAPE)) 
    model.add(Conv2D(32, kernel_size=(5, 5), activation='relu'))
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(OUT_SIZE))

    if verbose:
      model.summary()

    model.compile('adam', 'mse')

    return model

def get_model_v2(verbose=False):
    model = Sequential()
    model.add(Conv2D(8, kernel_size=(5, 5), activation='relu', input_shape=INPUT_SHAPE))
    model.add(BatchNormalization())
    model.add(Conv2D(16, kernel_size=(5, 5), activation='relu'))
    model.add(Conv2D(16, kernel_size=(5, 5), activation='relu'))
    model.add(Conv2D(16, kernel_size=(5, 5), activation='relu'))
    model.add(Flatten())
    model.add(Dropout(0.25))
    model.add(Dense(OUT_SIZE))

    model.summary()
    model.compile('adam', 'mse')

    return model

In [ ]:
print(INPUT_SHAPE)
mdl = get_model_v2()

In [ ]:
mdl.fit(x=itlseq, batch_size=128, epochs=100)

In [ ]:
import cv2
from google.colab.patches import cv2_imshow
def visualize(path, idx):
  infer = load_img(path)
  cv2img=load_img(path)
  cv2_imshow(cv2img)
  cv2img=cv2.cvtColor(cv2img, cv2.COLOR_GRAY2BGR)
  batch = np.zeros((1,*INPUT_SHAPE), dtype=np.float64)
  batch[0] = infer
  x,y,dx,dy=mdl.predict_on_batch(batch)[0]

  ox, oy, odx, ody = operators[idx]
  cv2.rectangle(cv2img, (int(x), int(y)), (int(x+dx), int(y+dy)), [0,0,255], 3)
  cv2.rectangle(cv2img, (int(ox), int(oy)), (int(ox+odx),int(oy+ody)), [0,255,0], 3)
  cv2_imshow(cv2img)
  print(idx)
  print('Infer: ', x,y,dx,dy)
  print('Ori: ', ox, oy, odx, ody)




In [ ]:
for j in range(12):
  visualize(f'output_proper/eq{j}.png', j)

In [ ]:
mdl.save('itl_op')